<tr width="140%" style="border: none;">
 <td style="border: none;" width="120px">
    <img align="left" src="https://i.imgur.com/zC3NxsV.png" width="130"/> 
  </td>
  <td style="border: none; text-align: left" width="80%">
    <h1><a>Exemplo Prático em Sistemas de Recomendação</a></h1>
    <td>
</tr>
<tr width="100%" style="border: none; text-align: left">
    <td colspan="2" style="border: none; text-align: left">
        <body> Leandro Bataglia, Pedro Machado, Gabriel Marinho, <br> Filipe Braida, Juliana Zamith, Leandro Alvim e Raul Ferreira </body>
    </td>
    </tr>

In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("CSV"); Pkg.add("DataFrames")
using CSV, DataFrames, Base.Threads, BenchmarkTools

path = "data/"
movies = CSV.File(path * "movies.csv") |> DataFrame;

 Activating environment at `~/Área de Trabalho/GitHub-projetos/multithreading_ERAD-RJ-2020/Project.toml`
   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
No Changes to `~/Área de Trabalho/GitHub-projetos/multithreading_ERAD-RJ-2020/Project.toml`
No Changes to `~/Área de Trabalho/GitHub-projetos/multithreading_ERAD-RJ-2020/Manifest.toml`
  Resolving package versions...
No Changes to `~/Área de Trabalho/GitHub-projetos/multithreading_ERAD-RJ-2020/Project.toml`
No Changes to `~/Área de Trabalho/GitHub-projetos/multithreading_ERAD-RJ-2020/Manifest.toml`


## Introdução
Neste notebook, iniciaremos a construção do nosso toy exemplo, que consiste em um simples sistema de recomendação para que possamos pôr em prática o que foi aprendido anteriormente durante o desenvolvimento de uma simples aplicação que se aproxima à realidade, mas com algumas simplificações para que possa ser implementada neste pequeno período de tempo que temos.

## O sistema de recomendação
Sistemas de recomendação, também chamados de sistemas de filtragem, têm como finalidade ofertar ao usuário do sistema uma coletânea de itens a partir da busca feita por este usuário dentro do sistema. Estas aplicações podem ser **personalizadas**, assim como as sugestões de aplicativos como Spotify e Youtube Music, que tentam gerar sugestões **de acordo com o perfil do usuário**, ou **não personalizadas** como o IMDB, que gera suas sugestões a partir de um *global score* associado aos itens.

Em nosso toy exemplo, nós iremos criar um sistema que efetua uma filtragem personalizada. Dentro deste ambiente, podem encontrar-se duas técnicas bem conhecidas, sendo elas a **filtragem baseada em conteúdo** e a **filtragem colaborativa**. A filtragem baseada em conteúdo tem seu processo de busca de itens feito com base nos meta-dados dos itens, enquanto que, na filtragem colaborativa, a busca é feita usando a semelhança entre os usuários ou itens como base.

## Implementação

Durante a implementação, nós iremos usar o algoritmo *nearest neighbors*, junto com o cálculo de similaridade do cosseno. Neste caso, vamos considerar que a matriz de notas pelos usuários não é uma matriz esparsa para uma simplificação do problema.

### Nearest Neighbors
Neste algorítimo é usada uma métrica determinada de acordo com o problema para verificar a distância, ou a similaridade entre os itens dentro de um mesmo espaço, definindo os vizinhos mais próximos do elemento alvo que em nosso caso irá ser um filme e os vizinhos mais próximos serão os filmes mais semelhantes.

<img src="https://miro.medium.com/max/650/1*OyYyr9qY-w8RkaRh2TKo0w.png" style="width: 500px"/>


### Similaridade do cosseno
Está métrica de similaridade foi escolhida pelo fato de que dentro deste problema a distância euclideana iria gerar números muito próximos pelo fato da matriz ser esparsa e de grande dimensão.

<img src="https://miro.medium.com/max/686/1*dGWOzgAYv9NUkWvkETQUTQ.png" style="width: 350px"/>

## Importação dos dados

In [2]:
ratings = CSV.File(path * "ratings.csv") |> DataFrame;

In [3]:
usersSize = length(unique(ratings[:, 1]))
moviesSize = size(movies, 1)

9742

## Construção das funções para acesso aos dados

In [4]:
getMovieNameById(id) = movies[findfirst(x -> x == id, movies[:, 1]), 2]::String
getMoviePositionById(id) = findfirst(x -> x == id, movies[:, 1])::Int64
getMovieIdByName(name) = movies[findfirst(x -> x == name, movies[:, 2]), 1]::Int64

getMovieIdByName (generic function with 1 method)

In [5]:
getMovieNameById(4)

"Waiting to Exhale (1995)"

In [6]:
getMoviePositionById(40)

37

In [7]:
movieRatings = zeros(usersSize, moviesSize)
for i=1:size(ratings, 1)
    movieRatings[ratings[i, 1], getMoviePositionById(ratings[i, 2])] = ratings[i, 3]
end

In [8]:
getGenres(id::Int) = movies[findfirst(x -> x == id, movies[:, 1]), 3]
getGenres(name::String) =  movies[findfirst(x -> x == name, movies[:, 2]), 3]

getGenres (generic function with 2 methods)

## Construção das funções para acesso aos dados

In [9]:
cosineDistance(v1, v2) = sum(v1 .* v2) / ((sum(v1 .^ 2) ^ (1/2)) * (sum(v1 .^ 2) ^ (1/2)))

cosineDistance (generic function with 1 method)

In [18]:
function similarity(id, k)
    position = getMoviePositionById(id)
#     println("Target: " * getMovieNameById(id))
    similarities = []
    for i=1:size(movieRatings, 2)
        push!(similarities, [cosineDistance(movieRatings[:, position], movieRatings[:, i]), movies[i, 2]])
    end
    sort!(similarities, by = x -> x[1], rev=true)
    deleteat!(similarities, findfirst(x -> x[2] == getMovieNameById(id), similarities))
#     for movie in similarities[1:k]
#         println("Similaridade: " * string(round(movie[1], digits=4)), " | " * movie[2])
#     end
    return nothing
end

similarity (generic function with 1 method)

In [19]:
similarity(2, 5)

In [20]:
@benchmark similarity(2,5)

BenchmarkTools.Trial: 
  memory estimate:  238.40 MiB
  allocs estimate:  173338
  --------------
  minimum time:     90.779 ms (14.88% GC)
  median time:      100.074 ms (17.57% GC)
  mean time:        104.667 ms (16.77% GC)
  maximum time:     175.882 ms (13.87% GC)
  --------------
  samples:          48
  evals/sample:     1

In [28]:
# A ideia aqui é que cada uma das threads percorra uma parte do catálogo de filmes, e daí esses vetores
# são concatenados em um único vetor no escopo acima
function similarity_threads(id, k)
    position = getMoviePositionById(id)
    # println("Target: " * getMovieNameById(id))
    all_similarities = [] #uma variável global um escopo acima do closure das threads
    div = divrem(size(movieRatings, 2), nthreads())
    len, rem = div[1], div[2]
    a = ReentrantLock()
    @threads for id = 1:nthreads()
        similarities = [] #local
        for i = (1:len) .+ (id-1)*len
            push!(similarities, [cosineDistance(movieRatings[:, position], movieRatings[:, i]), movies[i, 2]])
        end
        lock(a)
        all_similarities = Base.vcat(all_similarities, similarities)
        unlock(a)
    end
    for i in size(movieRatings,2)-rem+1:size(movieRatings, 2)
        push!(all_similarities, [cosineDistance(movieRatings[:, position], movieRatings[:, i]), movies[i, 2]])
    end
    sort!(all_similarities, by = x -> x[1], rev=true)
    deleteat!(all_similarities, findfirst(x -> x[2] == getMovieNameById(id), all_similarities))
#     for movie in all_similarities[1:k]
#         println("Similaridade: " * string(round(movie[1], digits=4)), " | " * movie[2])
#     end
    return nothing
end

similarity_threads (generic function with 1 method)

In [22]:
similarity_threads(2, 5)

In [23]:
@benchmark similarity_threads(2,5)

BenchmarkTools.Trial: 
  memory estimate:  238.36 MiB
  allocs estimate:  163675
  --------------
  minimum time:     71.487 ms (12.32% GC)
  median time:      84.609 ms (14.07% GC)
  mean time:        89.889 ms (14.43% GC)
  maximum time:     141.632 ms (11.94% GC)
  --------------
  samples:          56
  evals/sample:     1

In [ ]:
function similarity_threads!(A, movies, movieRatings, id)
    position = getMoviePositionById(id)
    #println("Target: " * getMovieNameById(id))
    div = divrem(size(movieRatings, 2), nthreads())
    len, rem = div[1], div[2]
    @threads for thread_id = 1:nthreads()
        for i = (1:len) .+ (threadid()-1)*len
            A[i] = (cosineDistance(movieRatings[:, position], movieRatings[:, i]), movies[i, 1])
        end
    end
    for i in size(movieRatings,2)-rem+1:size(movieRatings, 2)
            A[i] = (cosineDistance(movieRatings[:, position], movieRatings[:, i]), movies[i, 1])
    end
    sort!(A, by = x -> x[1], rev=true)
    A
end
@code_warntype similarity_threads!(result, movies, movieRatings, 2)

In [25]:
result = Vector{Tuple{Float64, Int64}}(undef, size(movies[:,1]))
@benchmark similarity_threads!(result, movies, movieRatings, 2)

BenchmarkTools.Trial: 
  memory estimate:  236.04 MiB
  allocs estimate:  105719
  --------------
  minimum time:     59.978 ms (10.06% GC)
  median time:      80.943 ms (12.55% GC)
  mean time:        81.436 ms (13.11% GC)
  maximum time:     111.441 ms (17.42% GC)
  --------------
  samples:          62
  evals/sample:     1

In [31]:
function similarity_threads_opt!(A::Vector{Tuple{Float64,Int64}}, movies::DataFrame, movieRatings::Array, id::Int64)
    position::Int64 = getMoviePositionById(id)
    #println("Target: " * getMovieNameById(id))
    div = divrem(size(movieRatings, 2), nthreads())
    len, rem = div[1], div[2]
    @threads for thread_id = 1:nthreads()
        for i = (1:len) .+ (threadid()-1)*len
            @inbounds A[i] = (cosineDistance(view(movieRatings,:, position), view(movieRatings,:, i)), movies[i, 1]::Int64)::Tuple{Float64,Int64}
        end
    end
    for i in size(movieRatings,2)-rem+1:size(movieRatings, 2)
            @inbounds A[i] = (cosineDistance(view(movieRatings,:, position), view(movieRatings,:, i)), movies[i, 1]::Int64)::Tuple{Float64,Int64}
    end
    sort!(A, by = x -> x[1], rev=true)
#    deleteat!(A, findfirst(x -> x[2] == id, A))
end
# @code_warntype similarity_threads_opt!(result, movies, movieRatings, 2)

similarity_threads_opt! (generic function with 1 method)

In [32]:
result = Vector{Tuple{Float64, Int64}}(undef, size(movies[:,1]))
@benchmark similarity_threads_opt!(result, movies, movieRatings, 2)

BenchmarkTools.Trial: 
  memory estimate:  141.36 MiB
  allocs estimate:  47778
  --------------
  minimum time:     38.676 ms (0.00% GC)
  median time:      56.241 ms (11.01% GC)
  mean time:        56.466 ms (14.39% GC)
  maximum time:     96.486 ms (10.30% GC)
  --------------
  samples:          89
  evals/sample:     1

In [ ]:
function similarity_threads_what!(A::Vector{Tuple{Float64,Int64}}, movies, movieRatings, id)
    position::Int64 = getMoviePositionById(id)
    moviesid = view(movies,:,1)
    #println("Target: " * getMovieNameById(id))
    div = divrem(size(movieRatings, 2), nthreads())
    len, rem = div[1], div[2]
    @threads for thread_id = 1:nthreads()
        for i = (1:len) .+ (threadid()-1)*len
            @inbounds A[i] = (cosineDistance(view(movieRatings,:, position), view(movieRatings,:, i)), moviesid[i]::Int64)::Tuple{Float64,Int64}
            #println((i, A[i]))
        end
    end
    for i in size(movieRatings,2)-rem+1:size(movieRatings, 2)
            @inbounds A[i] = (cosineDistance(view(movieRatings,:, position), view(movieRatings,:, i)), moviesid[i]::Int64)::Tuple{Float64,Int64}
    end
    sort!(A, by = x -> x[1], rev=true)
#   deleteat!(A, findfirst(x -> x[2] == id, A))
end
# @code_warntype similarity_threads_what!(result, movies, movieRatings, 2)

In [37]:
@benchmark similarity_threads_what!(result, movies, movieRatings, 2)

BenchmarkTools.Trial: 
  memory estimate:  141.07 MiB
  allocs estimate:  29259
  --------------
  minimum time:     32.255 ms (0.00% GC)
  median time:      49.539 ms (11.86% GC)
  mean time:        51.204 ms (12.53% GC)
  maximum time:     94.146 ms (6.27% GC)
  --------------
  samples:          98
  evals/sample:     1

In [ ]:
function movies_printer(movies, similarities, range::UnitRange)
    println("Filmes recomendados para você a partir do filme escolhido:")
    for i in range
        position = getMoviePositionById(similarities[i][2])
        println(movies[position, 2], " relevância [$(round(similarities[i][1] * 100))%]")
    end
end
movies_printer(movies, result, 1:10)